In [1]:
import copy
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import time

import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

# PyTorch
In this notebook you will gain some hands-on experience with [PyTorch](https://pytorch.org/), one of the major frameworks for deep learning. To install PyTorch. follow [the official installation instructions](https://pytorch.org/get-started/locally/). Make sure that you select the correct OS & select the version with CUDA if your computer supports it.
If you do not have an Nvidia GPU, you can install the CPU version by setting `CUDA` to `None`.
However, in this case we recommend using [Google Colab](https://colab.research.google.com/).
Make sure that you enable GPU acceleration in `Runtime > Change runtime type`.

You will start by re-implementing some common features of deep neural networks (dropout and batch normalization) and then implement a very popular modern architecture for image classification (ResNet) and improve its training loop.

# 1. Dropout
Dropout is a form of regularization for neural networks. It works by randomly setting activations (values) to 0, each one with equal probability `p`. The values are then scaled by a factor $\frac{1}{1-p}$ to conserve their mean.

Dropout effectively trains a pseudo-ensemble of models with stochastic gradient descent. During evaluation we want to use the full ensemble and therefore have to turn off dropout. Use `self.training` to check if the model is in training or evaluation mode.

Do not use any dropout implementation from PyTorch for this!

In [2]:
class Dropout(nn.Module):
    """
    Dropout, as discussed in the lecture and described here:
    https://pytorch.org/docs/stable/nn.html#torch.nn.Dropout
    
    Args:
        p: float, dropout probability
    """
    def __init__(self, p):
        super().__init__()
        self.p = p
        
    def forward(self, input):
        """
        The module's forward pass.
        This has to be implemented for every PyTorch module.
        PyTorch then automatically generates the backward pass
        by dynamically generating the computational graph during
        execution.
        
        Args:
            input: PyTorch tensor, arbitrary shape

        Returns:
            PyTorch tensor, same shape as input
        """
        
        # TODO: Set values randomly to 0.
        size = input.numel()
        stride = np.floor(1/self.p)
        count = 0
        for i in range(size):
            input[int(i*stride)] = 0
            count += 1
            if count == np.floor(size*self.p):
                break
        input[np.where(input!=0)] = input[np.where(input!=0)] / (1-self.p)
        return input
        

In [3]:
# Test dropout
test = torch.ones(10_000)
dropout = Dropout(0.5)
test_dropped = dropout(test)

# These assertions can in principle fail due to bad luck, but
# if implemented correctly they should almost always succeed.
assert np.isclose(test_dropped.sum().item(), 10_000, atol=400)
assert np.isclose((test_dropped > 0).sum().item(), 5_000, atol=200)

# 2. Batch normalization
Batch normalization is a trick use to smoothen the loss landscape and improve training. It is defined as the function
$$y = \frac{x - \mu_x}{\sigma_x + \epsilon} \cdot \gamma + \beta$$,
where $\gamma$ and $\beta$ and learnable parameters and $\epsilon$ is a some small number to avoid dividing by zero. The Statistics $\mu_x$ and $\sigma_x$ are taken separately for each feature. In a CNN this means averaging over the batch and all pixels.

Do not use any batch normalization implementation from PyTorch for this!

In [4]:
class BatchNorm(nn.Module):
    """
    Batch normalization, as discussed in the lecture and similar to
    https://pytorch.org/docs/stable/nn.html#torch.nn.BatchNorm1d
    
    Only uses batch statistics (no running mean for evaluation).
    Batch statistics are calculated for a single dimension.
    Gamma is initialized as 1, beta as 0.
    
    Args:
        num_features: Number of features to calculate batch statistics for.
    """
    def __init__(self, num_features):
        super().__init__()
        
        # TODO: Initialize the required parameters
        self.gamma = nn.Parameter(torch.ones(num_features,1),requires_grad=False)
        self.beta = nn.Parameter(torch.zeros(num_features,1),requires_grad=False)
        
    def forward(self, input):
        """
        Batch normalization over the dimension C of (N, C, L).
        
        Args:
            input: PyTorch tensor, shape [N, C, L]
            
        Return:
            PyTorch tensor, same shape as input
        """
        eps = 1e-5

        # TODO: Implement the required transformation
        num_N = input.size(0)
        input = input.permute(1,0,2)
        input = np.reshape(input,(2,-1))
        
        input_mean = input.mean(axis=1)
        input_std = torch.std(input, dim=1, unbiased=False)
        input_mean = input_mean[:,None]
        input_std = input_std[:,None]
        
        output= torch.mul(((input-input_mean)/(input_std+eps)), self.gamma) + self.beta

        output = np.reshape(output,(2,num_N,-1)) 
        output = output.permute(1,0,2)
 
        return output
        

In [5]:
# Tests the batch normalization implementation
torch.random.manual_seed(42)
test = torch.randn(8, 2, 4)

b1 = BatchNorm(2)
test_b1 = b1(test)

b2 = nn.BatchNorm1d(2, affine=False, track_running_stats=False)
test_b2 = b2(test)
assert torch.allclose(test_b1, test_b2, rtol=0.02)

# 3. ResNet
ResNet is the models that first introduced residual connections (a form of skip connections). It is a rather simple, but successful and very popular architecture. In this part of the exercise we will re-implement it step by step.

Note that there is also an [improved version of ResNet](https://arxiv.org/abs/1603.05027) with optimized residual blocks. Here we will implement the [original version](https://arxiv.org/abs/1512.03385) for CIFAR-10. Your dropout and batchnorm implementations won't help you here. Just use PyTorch's own layers.

This is just a convenience function to make e.g. `nn.Sequential` more flexible. It is e.g. useful in combination with `x.squeeze()`.

In [6]:
class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

We begin by implementing the residual blocks. The block is illustrated by this sketch:

![Residual connection](img/residual_connection.png)

Note that we use 'SAME' padding, no bias, and batch normalization after each convolution. You do not need `nn.Sequential` here. The skip connection is already implemented as `self.skip`. It can handle different strides and increases in the number of channels.

In [7]:
class ResidualBlock(nn.Module):
    """
    The residual block used by ResNet.
    
    Args:
        in_channels: The number of channels (feature maps) of the incoming embedding
        out_channels: The number of channels after the first convolution
        stride: Stride size of the first convolution, used for downsampling
    """
    
    def __init__(self, in_channels, out_channels, stride=1):
        super().__init__()        
        if stride > 1 or in_channels != out_channels:
            # Add strides in the skip connection and zeros for the new channels.
            self.skip = Lambda(lambda x: F.pad(x[:, :, ::stride, ::stride],
                                               (0, 0, 0, 0, 0, out_channels - in_channels),
                                               mode="constant", value=0))
        else:
            self.skip = nn.Sequential()
            
        # TODO: Initialize the required layers
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, bias = False, padding=1)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, bias = False, padding=1)
        self.batchnorm = nn.BatchNorm2d(out_channels,track_running_stats=False)
        
    def forward(self, input):
        # TODO: Execute the required layers and functions
        
        residual = self.conv1(input)
        residual = self.batchnorm(residual)
        residual = F.relu(residual)
        residual = self.conv2(residual)
        residual = self.batchnorm(residual)
        output = F.relu(residual + self.skip(input))
        return output
        
        

Next we implement a stack of residual blocks for convenience. The first layer in the block is the one changing the number of channels and downsampling. You can use `nn.ModuleList` to use a list of child modules.

In [8]:
class ResidualStack(nn.Module):
    """
    A stack of residual blocks.
    
    Args:
        in_channels: The number of channels (feature maps) of the incoming embedding
        out_channels: The number of channels after the first layer
        stride: Stride size of the first layer, used for downsampling
        num_blocks: Number of residual blocks
    """
    
    def __init__(self, in_channels, out_channels, stride, num_blocks):
        super().__init__()
        
        # TODO: Initialize the required layers (blocks)
        
        self.stack = nn.ModuleList([ResidualBlock(in_channels, out_channels, stride)]) 
        #self.stack.extend([ResidualBlock(in_channels, out_channels, stride)])
        self.stack.extend([ResidualBlock(out_channels, out_channels) for i in range(num_blocks-1)])
        
        """
        self.stack = nn.ModuleList([ResidualBlock(in_channels,out_channels,stride)])
        for i in range (num_blocks - 1):
            self.stack.append(ResidualBlock(out_channels,out_channels))
        """
        
   
    def forward(self, input):
        # TODO: Execute the layers (blocks)
        for m in self.stack:
            input = m(input)  
        return input
        

Now we are finally ready to implement the full model! To do this, use the `nn.Sequential` API and carefully read the following paragraph from the paper (Fig. 3 is not important):

![ResNet CIFAR10 description](img/resnet_cifar10_description.png)

Note that a convolution layer is always convolution + batch norm + activation (ReLU), that each ResidualBlock contains 2 layers, and that you might have to `squeeze` the embedding before the dense (fully-connected) layer.

In [9]:
n = 5
num_classes = 10

# TODO: Implement ResNet via nn.Sequential
#modules = []
#modules.appen(nn.Conv2d(3, 32*32, kernel_size=3, stride=stride, bias = False))

resnet = nn.Sequential(
    #pic size 32*32
    Lambda(lambda x: x.view(-1, 3, 32, 32)),
    #one extra layer
    nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
    nn.BatchNorm2d(16,track_running_stats=False),
    nn.ReLU(),

    ResidualStack(16, 16, stride=1, num_blocks=n), #output-shape: 128*16*32*32
    ResidualStack(16, 32, stride=2, num_blocks=n), #pic size 16*16
    ResidualStack(32, 64, stride=2, num_blocks=n), #pic size 8*8
    
    nn.AdaptiveAvgPool2d(1),
    Lambda(lambda x: x.squeeze()),
    nn.Linear(64, num_classes),
    #nn.Softmax(dim=1),
    
)


Next we need to initialize the weights of our model.

In [10]:
def initialize_weight(module):
    if isinstance(module, (nn.Linear, nn.Conv2d)):
        nn.init.kaiming_normal_(module.weight, nonlinearity='relu')
    elif isinstance(module, nn.BatchNorm2d):
        nn.init.constant_(module.weight, 1)
        nn.init.constant_(module.bias, 0)
        
resnet.apply(initialize_weight);

# 4. Training
So now we have a shiny new model, but that doesn't really help when we can't train it. So that's what we do next.

First we need to load the data. Note that we split the official training data into train and validation sets, because you must not look at the test set until you are completely done developing your model and report the final results. Some people don't do this properly, but you should not copy other people's bad habits.

In [11]:
class CIFAR10Subset(torchvision.datasets.CIFAR10):
    """
    Get a subset of the CIFAR10 dataset, according to the passed indices.
    """
    def __init__(self, *args, idx=None, **kwargs):
        super().__init__(*args, **kwargs)
        
        if idx is None:
            return
        
        self.data = self.data[idx]
        targets_np = np.array(self.targets)
        self.targets = targets_np[idx].tolist()

We next define transformations that change the images into PyTorch tensors, standardize the values according to the precomputed mean and standard deviation, and provide data augmentation for the training set.

In [12]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, 4),
    transforms.ToTensor(),
    normalize,
])
transform_eval = transforms.Compose([
    transforms.ToTensor(),
    normalize
])

In [13]:
ntrain = 45_000
train_set = CIFAR10Subset(root='./data', train=True, idx=range(ntrain),
                          download=True, transform=transform_train)
val_set = CIFAR10Subset(root='./data', train=True, idx=range(ntrain, 50_000),
                        download=True, transform=transform_eval)
test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                        download=True, transform=transform_eval)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [14]:
dataloaders = {}
dataloaders['train'] = torch.utils.data.DataLoader(train_set, batch_size=128,
                                                   shuffle=True, num_workers=2,
                                                   pin_memory=True)
dataloaders['val'] = torch.utils.data.DataLoader(val_set, batch_size=128,
                                                 shuffle=False, num_workers=2,
                                                 pin_memory=True)
dataloaders['test'] = torch.utils.data.DataLoader(test_set, batch_size=128,
                                                  shuffle=False, num_workers=2,
                                                  pin_memory=True)

Next we push the model to our GPU (if there is one).

In [15]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
resnet.to(device);

Next we define a helper method that does one epoch of training or evaluation. We have only defined training here, so you need to implement the necessary changes for evaluation!

In [16]:
def run_epoch(model, optimizer, dataloader, train):
    """
    Run one epoch of training or evaluation.
    
    Args:
        model: The model used for prediction
        optimizer: Optimization algorithm for the model
        dataloader: Dataloader providing the data to run our model on
        train: Whether this epoch is used for training or evaluation
        
    Returns:
        Loss and accuracy in this epoch.
    """
    # TODO: Change the necessary parts to work correctly during evaluation (train=False)   
    device = next(model.parameters()).device  
    epoch_loss = 0.0
    epoch_acc = 0.0
    #print("size of dataloader: ", len(dataloader))
    
    if train:
        model.train()
        # Iterate over data
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            #print(len(dataloader))

            # forward
            #with torch.set_grad_enabled(True):
            pred = model(xb)
            loss = F.cross_entropy(pred, yb)
            top1 = torch.argmax(pred, dim=1)
            ncorrect = torch.sum(top1 == yb)
            #print("correct number", ncorrect)
        
            # zero the parameter gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # statistics
            epoch_loss += loss.item()
            epoch_acc += ncorrect.item()
        
    else:
        model.eval()
        # Iterate over data
        for xb, yb in dataloader:
            xb, yb = xb.to(device), yb.to(device)
            #print(len(dataloader))

            # forward
            pred = model(xb)
            loss = F.cross_entropy(pred, yb)
            top1 = torch.argmax(pred, dim=1)
            ncorrect = torch.sum(top1 == yb)
            #print("correct number", ncorrect)

            # statistics
            epoch_loss += loss.item()
            epoch_acc += ncorrect.item()
 
    epoch_loss /= len(dataloader.dataset)
    epoch_acc /= len(dataloader.dataset)
    #print("loss: ", epoch_loss)
    return epoch_loss, epoch_acc

Next we implement a method for fitting (training) our model. For many models early stopping can save a lot of training time. Your task is to add early stopping to the loop (based on validation accuracy). Early stopping usually means exiting the training loop if the validation accuracy hasn't improved for `patience` number of steps. Don't forget to save the best model parameters according to validation accuracy. You will need `copy.deepcopy` and the `state_dict` for this.

In [17]:
def fit(model, optimizer, lr_scheduler, dataloaders, max_epochs, patience):
    """
    Fit the given model on the dataset.
    
    Args:
        model: The model used for prediction
        optimizer: Optimization algorithm for the model
        lr_scheduler: Learning rate scheduler that improves training
                      in late epochs with learning rate decay
        dataloaders: Dataloaders for training and validation
        max_epochs: Maximum number of epochs for training
        patience: Number of epochs to wait with early stopping the
                  training if validation loss has decreased
                  
    Returns:
        Loss and accuracy in this epoch.
    """
    
    best_acc = 0
    curr_patience = patience
    
    for epoch in range(max_epochs):
        train_loss, train_acc = run_epoch(model, optimizer, dataloaders['train'], train=True)
        lr_scheduler.step()
        print(f"Epoch {epoch + 1: >3}/{max_epochs}, train loss: {train_loss:.2e}, accuracy: {train_acc * 100:.2f}%")
        
        val_loss, val_acc = run_epoch(model, None, dataloaders['val'], train=False)
        print(f"Epoch {epoch + 1: >3}/{max_epochs}, val loss: {val_loss:.2e}, accuracy: {val_acc * 100:.2f}%")
        
        # TODO: Add early stopping and save the best weights (in best_model_weights)
        if val_acc > best_acc:
            curr_patience = patience
            best_acc = val_acc
            best_model_weights = copy.deepcopy(model.state_dict())
        else:
            curr_patience -= 1
            if curr_patience <= 0:
                print("early stopping")
                break
    
    model.load_state_dict(best_model_weights)

In most cases you should just use the Adam optimizer for training, because it works well out of the box. However, a well-tuned SGD (with momentum) will in most cases outperform Adam. And since the original paper gives us a well-tuned SGD we will just use that.

In [18]:
optimizer = torch.optim.SGD(resnet.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
lr_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)

# Fit model
fit(resnet, optimizer, lr_scheduler, dataloaders, max_epochs=200, patience=100)

Epoch   1/200, train loss: 1.54e-02, accuracy: 29.58%
Epoch   1/200, val loss: 1.25e-02, accuracy: 43.16%
Epoch   2/200, train loss: 1.10e-02, accuracy: 48.73%
Epoch   2/200, val loss: 9.80e-03, accuracy: 56.16%
Epoch   3/200, train loss: 8.82e-03, accuracy: 59.74%
Epoch   3/200, val loss: 8.44e-03, accuracy: 63.26%
Epoch   4/200, train loss: 7.42e-03, accuracy: 66.39%
Epoch   4/200, val loss: 7.56e-03, accuracy: 67.96%
Epoch   5/200, train loss: 6.42e-03, accuracy: 71.24%
Epoch   5/200, val loss: 6.97e-03, accuracy: 70.64%
Epoch   6/200, train loss: 5.68e-03, accuracy: 74.73%
Epoch   6/200, val loss: 6.26e-03, accuracy: 73.78%
Epoch   7/200, train loss: 5.17e-03, accuracy: 77.07%
Epoch   7/200, val loss: 5.55e-03, accuracy: 76.90%
Epoch   8/200, train loss: 4.76e-03, accuracy: 78.78%
Epoch   8/200, val loss: 5.61e-03, accuracy: 77.26%
Epoch   9/200, train loss: 4.47e-03, accuracy: 80.14%
Epoch   9/200, val loss: 5.52e-03, accuracy: 77.78%
Epoch  10/200, train loss: 4.25e-03, accuracy:

Epoch  78/200, val loss: 4.33e-03, accuracy: 84.60%
Epoch  79/200, train loss: 1.49e-03, accuracy: 93.28%
Epoch  79/200, val loss: 4.28e-03, accuracy: 84.28%
Epoch  80/200, train loss: 1.49e-03, accuracy: 93.32%
Epoch  80/200, val loss: 4.85e-03, accuracy: 82.72%
Epoch  81/200, train loss: 1.49e-03, accuracy: 93.19%
Epoch  81/200, val loss: 4.11e-03, accuracy: 84.74%
Epoch  82/200, train loss: 1.48e-03, accuracy: 93.38%
Epoch  82/200, val loss: 4.68e-03, accuracy: 83.94%
Epoch  83/200, train loss: 1.53e-03, accuracy: 92.98%
Epoch  83/200, val loss: 4.33e-03, accuracy: 84.86%
Epoch  84/200, train loss: 1.45e-03, accuracy: 93.50%
Epoch  84/200, val loss: 4.40e-03, accuracy: 84.42%
Epoch  85/200, train loss: 1.48e-03, accuracy: 93.36%
Epoch  85/200, val loss: 3.98e-03, accuracy: 85.36%
Epoch  86/200, train loss: 1.44e-03, accuracy: 93.65%
Epoch  86/200, val loss: 4.42e-03, accuracy: 83.90%
Epoch  87/200, train loss: 1.45e-03, accuracy: 93.44%
Epoch  87/200, val loss: 4.73e-03, accuracy: 8

Epoch 156/200, train loss: 7.87e-05, accuracy: 99.73%
Epoch 156/200, val loss: 4.61e-03, accuracy: 87.96%
Epoch 157/200, train loss: 7.55e-05, accuracy: 99.78%
Epoch 157/200, val loss: 4.61e-03, accuracy: 88.06%
Epoch 158/200, train loss: 7.28e-05, accuracy: 99.78%
Epoch 158/200, val loss: 4.61e-03, accuracy: 88.02%
Epoch 159/200, train loss: 7.21e-05, accuracy: 99.78%
Epoch 159/200, val loss: 4.61e-03, accuracy: 87.86%
Epoch 160/200, train loss: 6.65e-05, accuracy: 99.83%
Epoch 160/200, val loss: 4.61e-03, accuracy: 87.96%
Epoch 161/200, train loss: 6.97e-05, accuracy: 99.80%
Epoch 161/200, val loss: 4.61e-03, accuracy: 87.94%
Epoch 162/200, train loss: 6.80e-05, accuracy: 99.81%
Epoch 162/200, val loss: 4.62e-03, accuracy: 88.02%
Epoch 163/200, train loss: 6.76e-05, accuracy: 99.81%
Epoch 163/200, val loss: 4.61e-03, accuracy: 88.00%
Epoch 164/200, train loss: 6.71e-05, accuracy: 99.84%
Epoch 164/200, val loss: 4.62e-03, accuracy: 87.94%
Epoch 165/200, train loss: 6.41e-05, accuracy:

Once the model is trained we run it on the test set to obtain our final accuracy.
Note that we can only look at the test set once, everything else would lead to overfitting. So you _must_ ignore the test set while developing your model!

In [19]:
test_loss, test_acc = run_epoch(resnet, None, dataloaders['test'], train=False)
print(f"Test loss: {test_loss:.1e}, accuracy: {test_acc * 100:.2f}%")

Test loss: 3.6e-03, accuracy: 87.61%


That's almost what was reported in the paper (92.49%) and we didn't even train on the full training set.

# Optional task: Squeeze out all the juice!

Can you do even better? Have a look at [A Recipe for Training Neural Networks](https://karpathy.github.io/2019/04/25/recipe/) and some state-of-the-art architectures such as [EfficientNet architecture](https://ai.googleblog.com/2019/05/efficientnet-improving-accuracy-and.html). Play around with the possibilities PyTorch offers you and see how close you can get to the [state of the art on CIFAR-10](https://paperswithcode.com/sota/image-classification-on-cifar-10).